In [14]:
import keras
import tensorflow
import h5py
import numpy as np
import chowdhury_s_model_builder
import os

import tensorflow as tf
from keras.api.models import Sequential, Model, load_model
from keras.api.layers import Dense, Conv2D, Flatten, MaxPooling2D, Dropout, BatchNormalization, LSTM, Input, GlobalAveragePooling2D
from keras.api.optimizers import Adam
from keras import regularizers

import tensorflow as tf
print(tf.__version__)

from sklearn.metrics import classification_report, multilabel_confusion_matrix, ConfusionMatrixDisplay
from sklearn.metrics import precision_score, recall_score, f1_score
import matplotlib.pyplot as plt

2.19.0


In [15]:
with h5py.File("Data/Emotify_Annotated_Data.h5", "r") as hf:
    train_data = hf["train"][:]  # Load the train dataset
    label_data = hf["label"][:]  # Load the label dataset

print(train_data.shape, train_data.dtype)  # Check the shape and type
print(label_data.shape, label_data.dtype)

(400, 256, 1292) float32
(400, 7) int8


In [16]:
train_data = train_data[..., np.newaxis]

print(train_data.shape, train_data.dtype)  # Check the shape and type
print(label_data.shape, label_data.dtype)

(400, 256, 1292, 1) float32
(400, 7) int8


In [17]:
input_shape = train_data.shape[1], train_data.shape[2], train_data.shape[3]
print(input_shape)

(256, 1292, 1)


In [28]:
from sklearn.model_selection import train_test_split

# Assuming train_data and label_data are already loaded
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(train_data, label_data, test_size=0.9, random_state=42)

# Optionally, split the training set further into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25, random_state=42)  # 0.25 x 0.8 = 0.2

# Print the shapes to verify
print("Training set shape:", X_train.shape, y_train.shape)
print("Validation set shape:", X_val.shape, y_val.shape)
print("Test set shape:", X_test.shape, y_test.shape)

Training set shape: (30, 256, 1292, 1) (30, 7)
Validation set shape: (10, 256, 1292, 1) (10, 7)
Test set shape: (360, 256, 1292, 1) (360, 7)


In [29]:
model = load_model("model_epoch_52_acc_0.82_loss_0.02_valacc_0.91_valloss_0.02.keras")
model.summary()

/home/nigelchua0412/.local/lib/python3.12/site-packages/keras/src/saving/saving_lib.py:757: UserWarning: Skipping variable loading for optimizer 'adam', because it has 82 variables whereas the saved optimizer has 86 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 256, 1292, 1)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d (Conv2D)                 │ (None, 126, 644, 64)   │         1,664 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 126, 644, 64)   │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 126, 644, 64)   │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 126, 644, 64)   │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 63, 322, 64)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 63, 322, 64)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 63, 322, 128)   │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, 63, 322, 128)   │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_3 (Conv2D)               │ (None, 63, 322, 128)   │       147,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_3           │ (None, 63, 322, 128)   │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 31, 161, 128)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 31, 161, 128)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_4 (Conv2D)               │ (None, 31, 161, 256)   │       295,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_4           │ (None, 31, 161, 256)   │         1,024 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_5 (Conv2D)               │ (None, 31, 161, 256)   │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_5           │ (None, 31, 161, 256)   │         1,024 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_6 (Conv2D)               │ (None, 31, 161, 384)   │       885,120 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_6           │ (None, 31, 161, 384)   │         1,536 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_7 (Conv2D)               │ (None, 31, 161, 512)   │     1,769,984 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_7           │ (None, 31, 161, 512)   │         2,048 │
│ (BatchNormalization)            │                        │             

 Total params: 15,160,023 (57.83 MB)

 Trainable params: 5,051,975 (19.27 MB)

 Non-trainable params: 4,096 (16.00 KB)

 Optimizer params: 10,103,952 (38.54 MB)

In [30]:
model.compile(optimizer=Adam(learning_rate=3.1250e-05),
                           loss='binary_crossentropy', metrics=["accuracy"]
                           )

In [31]:
results = model.evaluate(train_data, label_data, batch_size=8, verbose=1)
print("Validation Loss:", results[0])
print("Validation Accuracy:", results[1])
# Predict the labels for the validation set

50/50 ━━━━━━━━━━━━━━━━━━━━ 6s 95ms/step - accuracy: 0.8458 - loss: 0.0513
Validation Loss: 0.047657910734415054
Validation Accuracy: 0.8424999713897705


In [23]:
# Assuming 'model' is your trained model
y_pred = model.predict(train_data)

# Convert probabilities to binary labels (e.g., threshold at 0.5)
y_pred_binary = (y_pred > 0.5).astype(int)

13/13 ━━━━━━━━━━━━━━━━━━━━ 5s 323ms/step


In [32]:

label_names = ['melody', 'articulation', 'rhythm_complexity', 'rhythm_stability', 'dissonance', 'atonality', 'mode']
# Generate the classification report
report = classification_report(label_data, y_pred_binary, target_names=label_names)
print(report)

# Generate the multilabel confusion matrix
mcm = multilabel_confusion_matrix(label_data, y_pred_binary)
print("Multilabel Confusion Matrix:")
print(mcm)

                   precision    recall  f1-score   support

           melody       0.95      0.99      0.97       103
     articulation       0.98      0.98      0.98        88
rhythm_complexity       0.85      0.89      0.87        19
 rhythm_stability       0.95      0.95      0.95        81
       dissonance       0.89      0.89      0.89        27
        atonality       0.97      1.00      0.98       208
             mode       0.94      1.00      0.97        67

        micro avg       0.96      0.98      0.97       593
        macro avg       0.93      0.96      0.94       593
     weighted avg       0.96      0.98      0.97       593
      samples avg       0.90      0.91      0.90       593

Multilabel Confusion Matrix:
[[[292   5]
  [  1 102]]

 [[310   2]
  [  2  86]]

 [[378   3]
  [  2  17]]

 [[315   4]
  [  4  77]]

 [[370   3]
  [  3  24]]

 [[186   6]
  [  1 207]]

 [[329   4]
  [  0  67]]]


/home/nigelchua0412/.local/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/nigelchua0412/.local/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/nigelchua0412/.local/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in samples with no true nor predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [33]:
import seaborn as sns
import matplotlib.pyplot as plt

# Define label names
label_names = ['melody', 'articulation', 'rhythm_complexity', 'rhythm_stability', 'dissonance', 'atonality', 'mode']

# Plot the confusion matrix for each label
for i, cm in enumerate(mcm):
    print(f'Confusion Matrix for {label_names[i]}')
    print('Actual', cm)
    print("Predicted")
    # plt.figure(figsize=(6, 4))
    # sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=['Predicted Negative', 'Predicted Positive'], yticklabels=['Actual Negative', 'Actual Positive'])
    # plt.title(f'Confusion Matrix for {label_names[i]}')
    # plt.xlabel('Predicted')
    # plt.ylabel('Actual')
    # plt.show()


Confusion Matrix for melody
Actual [[292   5]
 [  1 102]]
Predicted
Confusion Matrix for articulation
Actual [[310   2]
 [  2  86]]
Predicted
Confusion Matrix for rhythm_complexity
Actual [[378   3]
 [  2  17]]
Predicted
Confusion Matrix for rhythm_stability
Actual [[315   4]
 [  4  77]]
Predicted
Confusion Matrix for dissonance
Actual [[370   3]
 [  3  24]]
Predicted
Confusion Matrix for atonality
Actual [[186   6]
 [  1 207]]
Predicted
Confusion Matrix for mode
Actual [[329   4]
 [  0  67]]
Predicted


In [34]:
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
# import ace_tools as tools
# Define label names
label_names = ['melody', 'articulation', 'rhythm_complexity', 'rhythm_stability', 'dissonance', 'atonality', 'mode']

metrics = []

for i, cm in enumerate(mcm):
    tn, fp = cm[0]
    fn, tp = cm[1]
    precision = tp / (tp + fp) if (tp + fp) > 0 else 0
    recall = tp / (tp + fn) if (tp + fn) > 0 else 0
    f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
    accuracy = (tp + tn) / cm.sum()
    metrics.append({
        'label': label_names[i],
        'precision': precision,
        'recall': recall,
        'f1_score': f1,
        'accuracy': accuracy
    })

df_metrics = pd.DataFrame(metrics)
# Display metrics table
print(df_metrics)
# tools.display_dataframe_to_user(name="Label-wise Metrics", dataframe=df_metrics)

# # Plot all confusion matrices in a grid
# fig, axes = plt.subplots(2, 3, figsize=(15, 10))
# axes = axes.flatten()
# for i, cm in enumerate(cms):
#     sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
#                 xticklabels=['Pred Neg', 'Pred Pos'],
#                 yticklabels=['True Neg', 'True Pos'],
#                 ax=axes[i])
#     axes[i].set_title(label_names[i])
#     axes[i].set_xlabel('Predicted')
#     axes[i].set_ylabel('Actual')

# plt.tight_layout()
plt.show()

               label  precision    recall  f1_score  accuracy
0             melody   0.953271  0.990291  0.971429    0.9850
1       articulation   0.977273  0.977273  0.977273    0.9900
2  rhythm_complexity   0.850000  0.894737  0.871795    0.9875
3   rhythm_stability   0.950617  0.950617  0.950617    0.9800
4         dissonance   0.888889  0.888889  0.888889    0.9850
5          atonality   0.971831  0.995192  0.983373    0.9825
6               mode   0.943662  1.000000  0.971014    0.9900


In [35]:
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score

thresholds = [0.3, 0.4, 0.5, 0.6]
for t in thresholds:
    preds = (model.predict(train_data) > t).astype(int)
    precision = precision_score(label_data, preds, average='macro')
    recall = recall_score(label_data, preds, average='macro')
    f1 = f1_score(label_data, preds, average='macro')
    accuracy = accuracy_score(label_data, preds)
    print(f"\nThreshold: {t}")
    print(f"  Precision: {precision:.4f}")
    print(f"  Recall:    {recall:.4f}")
    print(f"  F1 Score:  {f1:.4f}")
    print(f"  Accuracy:  {accuracy:.4f}")


13/13 ━━━━━━━━━━━━━━━━━━━━ 5s 326ms/step

Threshold: 0.3
  Precision: 0.9043
  Recall:    0.9581
  F1 Score:  0.9300
  Accuracy:  0.9050
13/13 ━━━━━━━━━━━━━━━━━━━━ 4s 270ms/step

Threshold: 0.4
  Precision: 0.9294
  Recall:    0.9581
  F1 Score:  0.9434
  Accuracy:  0.9225
13/13 ━━━━━━━━━━━━━━━━━━━━ 4s 265ms/step

Threshold: 0.5
  Precision: 0.9336
  Recall:    0.9567
  F1 Score:  0.9449
  Accuracy:  0.9275
13/13 ━━━━━━━━━━━━━━━━━━━━ 4s 273ms/step

Threshold: 0.6
  Precision: 0.9404
  Recall:    0.9501
  F1 Score:  0.9451
  Accuracy:  0.9250
